**Run the codes in this sequence: A, B, C, D, E, F, G, H, I, J, K.**

# (A) Import libraries

In [1]:
# os.chdir(r'C:/Users/UWAdmin/Desktop/TEST2/line codes')

from pymeasure.instruments.lighthousephotonics import Sprout #import laser codes
laser_power = Sprout('COM4')
laser_power.adapter.connection.baud_rate = 19200
laser_power.adapter.connection.read_termination = '\r'
laser_power.adapter.connection.write_termination = '\r'
laser_power.power = 0.01


C:\Users\UWAdmin\Anaconda3\lib\site-packages\pyvisa\highlevel.py:3355: FutureWarning: get_instrument is deprecated and will be removed in 1.12, use open_resource instead.
  FutureWarning,


In [7]:
from threading import *
import time 
from datetime import timedelta, date

import rpy2.robjects as robjects

# from newportxps import NewportXPS #motion controller

import os

import numpy as np
import pandas as pd
from random import randrange
import csv

from flipper import mirror #import flipper mirror codes
from spectra import capture_photo, ration #import spectrometer codes



FileNotFoundError: Could not find module 'ftd2xx.dll' (or one of its dependencies). Try using the full path with constructor syntax.

# (B) Activate motion controller

In [77]:
XPSD_REMOTE_IP = '192.168.254.254'

xps = NewportXPS(XPSD_REMOTE_IP)
print(xps.status_report())

#pos_all() to get position of all axes
def pos_all():
    for sname, _ in xps.stages.items():
        print('{}: {}'.format(sname, xps.get_stage_position(sname)))
        

xps.kill_group('XYZ')
xps.initialize_allgroups()
xps.home_allgroups()

# XPS host:         192.168.254.254 (XPS-1B81)
# Firmware:         XPS-D-N13019
# Current Time:     Thu Apr 14 06:32:11 2022
# Last Reboot:      Thu Mar  3 02:07:55 2022
# Trajectory Group: XYZ
# Groups and Stages
XYZ (multipleaxesinuse), Status: Ready state from motion
   XYZ.X (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: First driver powered on - ZM low level
      Positioner Errors: OK
   XYZ.Y (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: First driver powered on - ZM low level
      Positioner Errors: OK
   XYZ.Z (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: First driver powered on - ZM low level
      Positioner Errors: OK


In [41]:
# xps.kill_group('XYZ')
# xps.initialize_allgroups()
# xps.home_allgroups()

pos_all()

XYZ.X: 6.7000016736
XYZ.Y: 12.5000065566
XYZ.Z: 1.2100017816


In [5]:
mirror("off")

MIRROR DOWN


## (C) Switch on the spectrometer

In [6]:
#Before running this code check that AddInProcess.exe is running (closed if open)
#Close anything related to Light Field

capture_photo("start",2,1,0) # the values 2,1,0 don't matter, they are later used for indexing tasks

# (L) To make sure that the material gets laser beam of max. intensity , focusing of z axis needs to be properly adjusted. this is done by adjust()

In [7]:
import time
def adjust(begin, end, step) -> tuple:
    line=1
    xps.move_stage('XYZ.X',5)
    xps.move_stage('XYZ.Y',7)
    zz=np.arange(begin,end+0.1,step)
    intensity=[]
    for i in range(len(zz)):

        xps.move_stage('XYZ.Z',zz[i])
        time.sleep(5)
        p=capture_photo('adjust',i,line,0)
        intensity.append(p)
    mirror('off')

    print(intensity)

    ix=intensity.index(np.max(intensity))
    xps.move_stage('XYZ.Z',zz[ix])
    print("adjusted z position is: ",zz[ix])
    return intensity, zz[ix]

In [ ]:
import time
import statistics
"""
A common issue withe adjust function is that it if there is a random spike
in the raman the program will take that at the inensity, this function seeks to
solve that by averaging out an average of 3 scans to dampen any
"""    

def adjust(begin, end, step) -> tuple:
    line=1
    xps.move_stage('XYZ.X',5)
    xps.move_stage('XYZ.Y',7)
    zz=np.arange(begin,end+0.1,step)
    intensity=[]
    for i in range(len(zz)):

        xps.move_stage('XYZ.Z',zz[i])
        time.sleep(5)
        p=capture_photo('adjust',i,line,0)
        intensity.append(p)
    mirror('off')

    print(intensity)

    ix=intensity.index(np.max(intensity))
    xps.move_stage('XYZ.Z',zz[ix])
    print("adjusted z position is: ",zz[ix])
    return intensity, zz[ix]

In [8]:
mirror('on')
laser_power.power = 0.01

MIRROR UP


In [10]:
#Check current position
pos_all()

XYZ.X: 4.999992003
XYZ.Y: 7.0000065036
XYZ.Z: 1.2100017816


## (D) Import pressure controls
#Used to input new Sample
#Move this before auto focusing

In [42]:
from pressure import close_valve,open_valve,close_all,current_pressure,gopr, to_ambient, to_vacuum
current_pressure()

12

In [43]:
from pymeasure.instruments.edwards import NXDS
import time
# connect to NXDS vacuum pump
pump = NXDS('ASRL5::INSTR')

In [69]:
open_valve("vac")
current_pressure()
time.sleep(3)
pump.start()
time.sleep(180)
pump.stop()
close_valve("vac")

In [70]:
close_valve("vac")
print(current_pressure())

0


In [71]:
open_valve("argon")
open_valve("mid")
time.sleep(1)
close_valve("argon")
close_valve("mid")
current_pressure()

33

In [72]:
to_vacuum(3)

0

In [62]:
mirror('on')

#Iterate for input to find z value
print("Enter z value (enter 0 when done):")
z=''
while z != 0:
    if z == '':
        z = float(input(':'))
    print(z)
    xps.move_stage('XYZ.Z', z)
    xps.move_stage('XYZ.X', 5)
    xps.move_stage('XYZ.Y', 7)
    z = float(input(':'))
    

# mirror('on')
pos_all()
while input('Is LightField Scan Stopped?:(yes/no)')!='yes':
    continue

MIRROR UP
Enter z value (enter 0 when done):
1.18
XYZ.X: 4.999992003
XYZ.Y: 7.0000065036
XYZ.Z: 1.1800012986


# (E) Prepare initial dataset

## MAKE SURE THE BOUNDS ARE COMMENSURATE WITH R CODE PARAM SET

In [17]:
#os.chdir(r'C:/Users/UWAdmin/Desktop/TEST2/Campaigns/2022-03-29/')

In [8]:
power=[]
line_time=[]
pressure=[]

print("Enter number of initial samples (min 4 (and even)): ")
NR_LINES = 7
print("Enter what experiment this is for the day of the date of:",str(date.today()))

exp_num = 1
#exp_num = input()

powr_range = (5, 1190)
tm_range = (1050,5000)
pr_range = (100,350)

for x in range(NR_LINES):
    powr = randrange(powr_range[0], powr_range[1], 1) # in mW
    tm = randrange(tm_range[0], tm_range[1], 1) # in ms
    pr = randrange(pr_range[0], pr_range[1], 10) # in psi
    for i in range(9):
        power.append(powr)
        line_time.append(tm)
        pressure.append(pr)

p=str(date.today())+"-Series"+str(exp_num)
print(f"Here is the new Campaign Folder:{p}")
path=r'C:\Users\hwahab\OneDrive - University of Wyoming\My files\Johnson Research Group\05-Project_data\00-MRS-multiobjective'
os.chdir(path)
os.mkdir(p)
os.chdir(p)

row=['power','time','pressure','ratio','resistance']
plot_row=['power','time','pressure','dummy_ratio', 'dummy_resistance', 'pred_mean', 'pred_upper', 'ei']
#rename this to raw_post_processed.csv
with open('batch-raw-data.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
        writer.writerows(zip(power, line_time, pressure))

#rename this to raw_pre-patterning.csv
with open('batch-raw-data-pre.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
        writer.writerows(zip(power, line_time, pressure))

# THIS HAS THREE ADDITIONAL COLUMNS
with open('batch-plot-data.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(plot_row)
        # writer.writerows(zip(power, line_time, pressure))

# os.chdir(path)

df2=pd.read_csv('batch-raw-data.csv')
df2=df2.drop_duplicates() #keep only the unique rows
df2.head()
df2.to_csv('batch-ai-data.csv',index=False) #this is what will be read by mlrBO in th R code
# df2.to_csv('plot_data.csv' index=False)

Enter number of initial samples (min 4 (and even)): 
Enter what experiment this is for the day of the date of: 2022-04-15
Here is the new Campaign Folder:2022-04-15-Series1


# (F) Defining threads

-Thread for Mirror 

-Thread for switching the mirror on and off

In [21]:
#Change this to motion_thread
class Aa(Thread):
    """MOTOR THREAD"""
    def run(self):
        import pandas as pd
        d = pd.read_csv('foreward.trj',header=None) # scantime 2 (see segment 1 or row1)
        d = d.loc[:, (d != 0).any(axis=0)] # remove last 3 cols with 0
        d.columns = ['ramptime','rampdist','rampvel']
        total_time=np.sum(d['ramptime'])
        
        if d['ramptime'][0]<0.5:
            time.sleep(.5-d['ramptime'][0])
        print("MOTION THREAD\n")
        print("XPS run trajectory")
        xps.run_trajectory('foreward',)
        print('Total time {} in class'.format(total_time))
        time.sleep(total_time)
        print("TOTAL_TIME DONE, LASER 0.05W")
        laser_power.power=0.05
        print("finished and current position is:\n")
        pos_all()

#Change this to mirror_thread
class Bb(Thread):
    """MOTOR THREAD"""
    def run(self):
        import pandas as pd
        d = pd.read_csv('foreward.trj',header=None) # scantime 2 (see segment 1 or row1)
        d = d.loc[:, (d != 0).any(axis=0)] # remove last 3 cols with 0
        d.columns = ['ramptime','rampdist','rampvel']
        mirror_sleep=d['ramptime'][0]-.5
        print("MIRROR THREAD")
        for i in range(2):
            if i==0:
                print(d['ramptime'][0])
                if d['ramptime'][0]<0.5:
                    print("mirror on")
       
                    start_time = time.monotonic()
                    mirror('on')
       
                    end_time = time.monotonic()
                    print(timedelta(seconds=end_time - start_time))
                    time.sleep(d['ramptime'][1]) #time for linear line    

                else:
                    print('{} > 0.5'.format(d['ramptime'][0]))
                    print('Mirror sleep {} in class'.format(mirror_sleep))
                    time.sleep(mirror_sleep)
                
                    print("mirror on")
                    start_time = time.monotonic()
                    mirror('on')
                    end_time = time.monotonic()
                    print(timedelta(seconds=end_time - start_time))
                    time.sleep(d['ramptime'][1]) #time for linear line
            else:
                mirror('off')
                print("mirror off")

# (G) Define coordinate details

**Takes input for**  
  
-number of lines

-starting position of x

-starting position of y

-vertical gap/step size

<!-- # (G) Define coordinate details

**Takes input for**  
**COLUMN 1**  
-number of lines

-starting position of x

-starting position of y

-vertical gap/step size

-**COLUMN 2** - yes/no

-starting position of x

-starting position of y

-vertical gap/step size -->

In [40]:
def get_move_y(lines: int, starty: int) -> list:
    move_y = [0 for i in range(int(lines))]
    for i in range(int(lines)):
    #     print(i)
        if i == 0: move_y[i] = float(starty)
        if i > 0 :
            move_y[i] = float(move_y[i-1]) + float(step_y)
    return move_y

In [4]:
# df = pd.read_csv("dataset.csv")
# df.head()

# print("Enter number of lines, starting point for x axis, y axis and step size for y axis:")
lines ,startx, starty, step_y = 7, 2.5, 6.5, 1

print(lines,startx,starty,step_y)
move_y = [0 for i in range(int(lines))]


for i in range(int(lines)):
#     print(i)
    if i == 0: move_y[i] = float(starty)
    if i > 0 :
        move_y[i] = float(move_y[i-1]) + float(step_y)
#         print(move_y[i-1]+float(step_y))


print("Lines will be made at following y axis:",move_y,"\n")


7 2.5 6.5 1
Lines will be made at following y axis: [6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12.5] 



# (H) Parameters predicted by BO is written on data.csv. This is further written on dataset.csv and dataset-pre.csv using write_more() function

In [23]:
def write_more() -> None:
    """
    Used after doing the AI stuff
    """
    d = pd.read_csv('data.csv')
    ln = d.shape[0]

    vpower = d['power'][ln-1]
    vtime = d['time'][ln-1]
    vpressure = d['pressure'][ln-1]

    d1 = pd.read_csv('dataset.csv')
    ln = d1.shape[0]
    d1.loc[ln,"power"] = vpower
    d1.loc[ln,"time"] = vtime
    d1.loc[ln,"pressure"] = vpressure
    d1.to_csv('dataset.csv',index = False)
    d1.to_csv('dataset-pre.csv',index = False)

# (I) Each line has 3 spots to check and each spot 3 times. so in the csv 9 rows of data for each line

**This is done using repeats() function**

In [78]:
def repeats() -> None:
    # d1 = pd.read_csv('data.csv')
    df2 = pd.read_csv('dataset.csv')
    ln = len(df2['power'])
    m = ln
    print(f"DATASET LENGTH: {ln}")
    counter  = m
    for i in range(8):
        toAdd = [df2['power'][m-1],df2['time'][m-1],df2['pressure'][m-1]]
        filename = "dataset.csv"
        with open(filename, "r") as infile:
            reader = list(csv.reader(infile))
            reader.insert(counter+1, toAdd)

        with open(filename, "w", newline='') as outfile:
            writer = csv.writer(outfile)
            for line in reader:
                writer.writerow(line)
                
    for i in range(8):
        toAdd = [df2['power'][m - 1],df2['time'][m - 1],df2['pressure'][m - 1]]
        filename = "dataset-pre.csv"
        with open(filename, "r") as infile:
            reader = list(csv.reader(infile))
            reader.insert(counter + 1, toAdd)

        with open(filename, "w", newline='') as outfile:
            writer = csv.writer(outfile)
            for line in reader:
                writer.writerow(line)

# (J) For the 9 readings of a single line, mean is taken. it's done by take_mean() function

In [2]:
SPOTS_MEASURED = 9

def take_mean(save_line: int) -> float:
    
    print(os.getcwd())
    df = pd.read_csv('dataset.csv')
    result = df['ratio'].iloc[-SPOTS_MEASURED:].mean()
    df['ratio'].iloc[-SPOTS_MEASURED:].fillna(0, inplace=True)
    df.to_csv('dataset.csv',index = False)

    df2 = pd.read_csv('data.csv')
    df2.loc[save_line,"ratio"] = result
    df2.to_csv('data.csv',index = False)

    return result

# (K) Begin the experiment

In [23]:
steps = 0
move_line, save_line = 0, 7

In [12]:
#path=r'C:\Users\UWAdmin\Desktop\TEST2\Campaigns\2022-04-11-Sample1'
#os.chdir(path)

In [5]:
col = 0
TOTAL_LINES = len(move_y)
steps = 0
for move_line in range(TOTAL_LINES):
        # print("For this print this is line #: ",move_line)
        # print("Experiment Line number: " ,move_line)
        
        save_line = col*TOTAL_LINES + move_line
        steps = save_line * 9
        print (f"before: {move_line}, {save_line}, {steps}")
        
        # print (f"after : {move_line}, {save_line}, {steps}")

before: 0, 0, 0
before: 1, 1, 9
before: 2, 2, 18
before: 3, 3, 27
before: 4, 4, 36
before: 5, 5, 45
before: 6, 6, 54


In [76]:
def main_batch_experiment(startx: float, move_y: list, col: int) -> None:
    """Starts the experiment. Currently works ok with equal length of col1 and col2 lines.
       startx: Starting position on the x-axis
       starty: Starting position on the y-axis
       step_y: The distance between lines on the y-axis
       move_y: List of positions on the y-axis
       col_no: Defines the column number"""

    # line_name_start = col_no*len(move_y) #0,1,   # What the first line # will be named
    # line_name_end = line_name_start + len(move_y) #What the last line # will be named
    
    import pandas as pd
    print(f'LOADING DATA.CSV FOR COLUMN: {col}') 
    df2 = pd.read_csv('batch-ai-data.csv')   # to avoid pd referenced before assignment
    print (df2)
    
    print("EXPERIMENT IS STARTING\n")
    import time
    time.sleep(5)
    # cc = 0
    # steps = 0
    MIRROR_MOVE_TIME = .5
    LINE_LENGTH = 2

    #set this to zero!!!!!!!
    #starting_line = 4
    STARTING_LINE = 0
    SPOTS_MEASURED = 9
    TOTAL_LINES = len(move_y) #+STARTING_LINE

    print(f"Here's the number of total lines:{TOTAL_LINES}")
    
    for move_line in range(STARTING_LINE,TOTAL_LINES):
        print(f"====== MOVE LINE: {move_line} ======")
        
        save_line = col * TOTAL_LINES + move_line
        steps = save_line * SPOTS_MEASURED
        print(f"====== SAVE LINE: {save_line} ======")
        #line = i

        # move the axes to their start position
        xps.move_stage('XYZ.Y',move_y[move_line])
        pos_all()
        print("\n\n")
        df2 = pd.read_csv('batch-ai-data.csv')
        # line = df2['ratio'].count() #find in batch-ai-data.csv what line number we're on
        
        # set time
        time_of_file = df2['time'][save_line]
        print("\nCurrent time is:",time_of_file)
        
        # set pressure
        pressure_of_file=df2['pressure'][save_line]
        for j in range(3):
            gopr(pressure_of_file)
            close_all()
            time.sleep(5)
        print("\n Pressure is now : ",pressure_of_file,"\n")

        xps.define_line_trajectories(start=float(startx),
                                 stop=float(startx)+LINE_LENGTH,
                                 step=0.01,
                                 scantime=(time_of_file/1000),
                                 axis='X')
        # downloads trajectory defined above
        xps.download_trajectory('foreward.trj')
        import pandas as pd
        d = pd.read_csv('foreward.trj', header=None) # scantime 2 (see segment 1 or row1)
        d = d.loc[:, (d != 0).any(axis=0)] # remove last 3 cols with 0
        d.columns = ['ramptime','rampdist','rampvel']
        print(d)

        total_time = np.sum(d['ramptime'])
        mirror_sleep = d['ramptime'][0] - MIRROR_MOVE_TIME
        print(f'Total time: {total_time}, Mirror_sleep: {mirror_sleep} in MAIN')

        # power will be set to assigned value, make sure mirror is OFF (!) \
        # and power is given enough time to reach it's value

        power_of_file = df2["power"][save_line]


    ###########################

        # print("#############STARTING PRE-MEASUREMENT ###########\n")
        # check=[float(startx) + .2*LINE_LENGTH, \
        #         float(startx) + .4*LINE_LENGTH, \
        #         float(startx) + .6*LINE_LENGTH]
        # # mm=0
        # for spot in range(3):
        #     print(f"########## spot: {spot} ##############")
        #     xps.move_stage('XYZ.X',check[spot])
        #     print(pos_all())

        #     for iii in range(3):
        #         print(f"############## iteration: {iii} ################")
        #         mirror("on")
        #         #writing G/D of 3 spots in 3 lines inside dataset.csv
        # #         capture_photo("on",spot,line)
        #         GD=capture_photo("first",spot,line,iii)

        #         mirror("off")
        #         time.sleep(5)

    ############################

    # PATTERN EXPERIMENT


        print("Power is now: {} mW".format(power_of_file))
        xps.move_stage('XYZ.X',(float(startx) - d['rampdist'][0]))
        pos_all()
        time.sleep(15)
        print("\n\n")
        mirror("off")
        a=power_of_file
        #laser_power.power=((a-5.7488)/0.138859)/1000
        laser_power.power = (a/0.0994)/1000
        time.sleep(15)
        
        
        print("###### Starting pattern in 5 seconds! #########")
        ##Draw Lines
        time.sleep(10)
        
        import sys
        a=Aa() # MOTION THREAD
        b=Bb() # MIRROR THREAD
        # stop_threads = False
        a.start()
        b.start()

        a.join()
        b.join()

        time.sleep(10)
        print("\n Job done\n")

        ##raman spectra analysis
        laser_power.power=0.05
        time.sleep(15)

    ########################################

    # post patterning raman analysis

        check=[float(startx) + .2*LINE_LENGTH, \
                float(startx) + .4*LINE_LENGTH, \
                float(startx) + .6*LINE_LENGTH]
        # mm=0
        for spot in range(3):
            print(f"############### SPOT {spot} ##################")
            xps.move_stage('XYZ.X',check[spot])
            print(pos_all())

            for iii in range(3):
                print(f"############# iteration {iii} ###############")
                mirror("on")
                #writing G/D of 3 spots in 3 lines inside dataset.csv
        #         capture_photo("on",spot,line)
                print("Measuring patterned line")
                GD = capture_photo("on", spot, save_line, iii)

                mirror("off")
                time.sleep(5)      

        if save_line <= TOTAL_LINES:
            print(f"############# LINE {save_line} <= {TOTAL_LINES}, CURRENT_STEP={steps}, TAKING MEAN: #############")
            take_mean(steps, save_line)

In [10]:
robjects.r('''
  experiment<-function(s){

  presToR <- function (x) {(x - 50) / 10}
  presToExp <- function (x) {x * 10 + 50}

  if(s==1){ #BO called first time to initialize and propose first point
  require(XML)

    presToR <- function (x) {(x - 50) / 10}
    presToExp <- function (x) {x * 10 + 50}

    suppressWarnings({suppressMessages({library(mlrMBO)
                                        library(ggplot2)
                                        library(doMC)
                                      })})

    ps = makeParamSet(
      makeIntegerParam("power", lower = 5, upper = 1190),
      makeIntegerParam("time", lower = 1050, upper = 5000),
      makeIntegerParam("pressure", lower = 5, upper = 30, trafo = function(x) x * 10 + 50)
    )

    ctrl = makeMBOControl(y.name = "resistance", propose.points = 7L)
    ctrl = setMBOControlInfill(ctrl, opt = "focussearch", 
                              opt.focussearch.maxit = 10,
                              opt.focussearch.points = 10000,
                              crit = makeMBOInfillCritEI())

    ctrl = setMBOControlMultiPoint(ctrl, method = "cl", cl.lie = min)
    ctrl = setMBOControlTermination(ctrl, iters = 1)

    data = read.csv("batch-ai-data.csv", colClasses = c(NA, NA, NA, "NULL", NA))
    data$pressure <- presToR(data$pressure)

    suppressMessages({opt.state = initSMBO(par.set = ps, 
                                          design = data,
                                          control = ctrl, 
                                          minimize = FALSE, 
                                          noisy = TRUE)})
    print("Proposed parameters:")
    prop = suppressWarnings({proposePoints(opt.state)})
    print(prop$prop.points)
    print(prop$crit.components)
    saveRDS(prop,'./prop.rds')
    saveRDS(opt.state,'./opt.state.rds')
    dummy_ratio = 0
    dummy_resistance = 0
    cat("Expected value (upper bound):\n")
    cat(paste(prop$crit.components$mean, " (",
              prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))

    prop$prop.points$pressure <- presToExp(prop$prop.points$pressure)

    x <- data.frame(prop$prop.points, dummy_ratio, dummy_resistance,
                    prop$crit.components$mean,
                    prop$crit.components$mean + prop$crit.components$se, 
                    2*prop$crit.components$mean + prop$crit.components$se)

    x2 <- c(prop$prop.points)

    write.table(x2, file = "batch-ai-data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

    write.table(x, file = "batch-plot-data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

  }
  else{
    
    presToR <- function (x) {(x-50)/10}
    presToExp <- function (x) {x*10+50}

    prop=readRDS('./prop.rds')
    opt.state=readRDS('./opt.state.rds')
    library(mlrMBO)
    library(R.utils)
    print(prop$crit.components)
    print(prop$prop.points)

    print("Proposed parameters:")
    l2keep <- 7
    nL <- countLines("batch-data.csv")
    s <- read.csv("batch-data.csv", header=FALSE, skip=nL-l2keep)['V5']
    updateSMBO(opt.state, x = prop$prop.points, y = as.list(s$V5))
    prop <- suppressWarnings({proposePoints(opt.state)})
    print(prop$prop.points)
    print(prop$crit.components)
    # saveRDS(prop,'./prop.rds')
    # saveRDS(opt.state,'./opt.state.rds')

    dummy_ratio = 0
    dummy_resistance = 0
    cat("Expected value (upper bound):\n")
    cat(paste(prop$crit.components$mean, " (",
              prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))

    prop$prop.points$pressure <- presToExp(prop$prop.points$pressure)

    x <- data.frame(prop$prop.points, dummy_ratio, dummy_resistance,
                    prop$crit.components$mean,
                    prop$crit.components$mean + prop$crit.components$se, 
                    prop$crit.components$mean + prop$crit.components$mean + prop$crit.components$se)
    x2<-c(prop$prop.points)

    write.table(x2, file = "batch-data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

    write.table(x, file = "batch-plot-data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

  }}

  ''')

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x00000247E7FD3A00> [RTYPES.CLOSXP]
R classes: ('function',)

In [12]:
rsum = robjects.r['experiment']
# rsum = robjects.r['test']
rsum((1))

R[write to console]: Error: package 'ParamHelpers' could not be loaded



RRuntimeError: Error: package 'ParamHelpers' could not be loaded


In [73]:
print(move_y)
todd_data = pd.read_csv('data.csv')
print(todd_data)
todd_data['ratio'].count()
move_y[4:7]
print(os.getcwd())
current_pressure()

[6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12.5]
    power  time  pressure     ratio
0     159  1958       100  0.857771
1     238  3549       220  1.333680
2    1079  2802       230  1.242668
3     724  2631       210  1.190802
4     481  3995       260  0.808519
5     839  4650       220  0.754259
6     278  3269       220  0.805314
7     823  4957       120  3.001912
8     887  5000       120  1.276346
9     816  4954       120       NaN
10    823  4957       120       NaN
11    823  4957       120       NaN
12    823  4957       120       NaN
13    823  4957       120       NaN
14    823  4957       120       NaN
C:\Users\UWAdmin\Desktop\TEST2\Campaigns\2022-04-13-Series1


1

In [75]:
import sys


# path=r'C:\Users\UWAdmin\Desktop\TEST2\Campaigns\2022-04-11-Sample1'
# os.chdir(path)

first_column = int(input("First column number:"))
first_column_verify = int(input("Verify first column number:"))
while first_column !=first_column_verify:
    first_column = int(input("Error, numbers don't match for first column. Re-enter first column number:"))
    first_column_verify = int(input("Verify first column number:"))




second_column = int(input("Second column number:"))
second_column_verify = int(input("Verify second column number:"))

while second_column !=second_column_verify:
    second_column = int(input("Error, numbers don't match for second column. Re-enter second column number:"))
    second_column_verify = int(input("Verify second column number:"))


intensity_check =capture_photo('adjust',0,0,0)
print("---Final Verification of Experiment---")
print("Current Pressure: ", current_pressure())
pos_all()
print("Checked Intesity: ",intensity_check)
print(f"Print to columns: {first_column} and {second_column}")



verify_start = input("Do you approve pressure, intensity, and column numbers (yes/no)")
if verify_start == "yes":
    "Pressure, intensity, and column numbers have been approved. Starting experiment..."
    startx1 = 2.2
    main_experiment(startx1, move_y, col=first_column)

    startx2 = 5.5
    main_experiment(startx2, move_y, col=second_column)
else:
    sys.exit()





MIRROR UP
 
MIRROR DOWN
---Final Verification of Experiment---
Current Pressure:  1
XYZ.X: 4.999992003
XYZ.Y: 7.0000065036
XYZ.Z: 1.1800012986
Current Intesity:  669.6666666666666
Print to columns: 2 and 3
LOADING DATA.CSV
    power  time  pressure     ratio
0     159  1958       100  0.857771
1     238  3549       220  1.333680
2    1079  2802       230  1.242668
3     724  2631       210  1.190802
4     481  3995       260  0.808519
5     839  4650       220  0.754259
6     278  3269       220  0.805314
7     823  4957       120  3.001912
8     887  5000       120  1.276346
9     816  4954       120       NaN
10    823  4957       120       NaN
11    823  4957       120       NaN
12    823  4957       120       NaN
13    823  4957       120       NaN
14    823  4957       120       NaN
EXPERIMENT IS STARTING

Here's the number of total lines:7
====== MOVE LINE: 0 ======
====== SAVE LINE: 14 ======
XYZ.X: 4.999992003
XYZ.Y: 6.5001754476
XYZ.Z: 1.1800012986




Current time is: 4957
do

C:\Users\UWAdmin\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\UWAdmin\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


     power time pressure
3638   823 4957        7
         se      mean
1 0.7966157 0.7514182
Expected value (upper bound):
0.751418202307775 (1.54803387345452)
136
#####################

LINE 14: DEFINING MBO EXPERIMENT
====== MOVE LINE: 1 ======
====== SAVE LINE: 15 ======
XYZ.X: 3.4000016418
XYZ.Y: 7.4999083572
XYZ.Z: 1.1800012986




Current time is: 4957
done
done
done

 Pressure is now :  120 

   ramptime  rampdist  rampvel
0  0.242082  0.097673  0.40347
1  4.981785  2.010000  0.40347
2  0.242082  0.097673  0.00000
Total time: 5.465948999999999, Mirror_sleep: -0.257918 in MAIN
Power is now: 823 mW
XYZ.X: 2.1026356218
XYZ.Y: 7.4999968542
XYZ.Z: 1.1800012986



MIRROR DOWN
###### Starting pattern in 5 seconds! #########
MIRROR THREAD
0.24208200000000002
mirror on
MIRROR UP
MOTION THREAD

XPS run trajectory
0:00:00.281000
MIRROR DOWN
mirror off
Total time 5.465948999999999 in class
TOTAL_TIME DONE, LASER 0.05W
finished and current position is:

XYZ.X: 4.307679972
XYZ.Y: 7.499996854

## Troubleshoot R code

In [ ]:
rsum=robjects.r['experiment']
rsum((1))
write_more()
repeats()

In [ ]:
line = 4
steps = 9

In [ ]:
rsum

In [ ]:
if line>3:
    mv=take_mean(steps,line)
    print(mv)
    steps+=9
    if mv is None:
        rsum((0))

    else:
        mv=float(mv)
        rsum((mv))
#     write_more()
#     repeats()

In [ ]:
col = 1
print(startx1,starty1,step_y1,move_y1,col)
# col = 1
print(startx2,starty2,step_y2,move_y2,col)

In [ ]:
os.getcwd()
rsum=robjects.r['sourin']
rsum((3.5))

In [ ]:
col = 0
main_experiment(startx1,starty1,step_y1,move_y1,col)

pre = "col2 "
[os.rename(f, "{}{}".format(pre, f)) for f in glob.glob("*foreground*")]

#call main_experiment for column 2
col = 1
main_experiment(startx2,starty2,step_y2,move_y2,col)

pre = "col3 "
[os.rename(f, "{}{}".format(pre, f)) for f in glob.glob("*foreground*")]

In [ ]:
#import time as timer
#start_time = timer.time()
#call main_experiment for column 1
col = 0
main_experiment(startx1,starty1,step_y1,move_y1,col)

pre = "col1 "
[os.rename(f, "{}{}".format(pre, f)) for f in glob.glob("*foreground*")]
        
#call main_experiment for column 2
# col = 1
# main_experiment(startx2,starty2,step_y2,move_y2,col)

# pre = "col2 "
# [os.rename(f, "{}{}".format(pre, f)) for f in glob.glob("*foreground*")]

#run_time = timer.time() - start_time

In [ ]:
x=2.5
tof=1050
length=2
xps.define_line_trajectories(start=float(x),
                                 stop=float(x)+length,
                                 step=0.01,
                                 scantime=(tof/1000),
                                 axis='X')
xps.download_trajectory('foreward.trj')
import pandas as pd
d = pd.read_csv('foreward.trj',header=None) # scantime 2 (see segment 1 or row1)
d = d.loc[:, (d != 0).any(axis=0)]
d.columns = ['ramptime','rampdist','rampvel']
d

# (G) Parameters predicted by BO is written on data.csv. This is further written on dataset.csv and dataset-pre.csv using write_more() function

In [ ]:
def write_more():
    d=pd.read_csv('data.csv')
    ln=d.shape[0]

    vpower=d['power'][ln-1]
    vtime=d['time'][ln-1]
    vpressure=d['pressure'][ln-1]

    d1=pd.read_csv('dataset.csv')
    ln=d1.shape[0]
    d1.loc[ln,"power"]=vpower
    d1.loc[ln,"time"]=vtime
    d1.loc[ln,"pressure"]=vpressure
    d1.to_csv('dataset.csv',index=False)
    d1.to_csv('dataset-pre.csv',index=False)

# (H) Each line has 3 spots to check and each spot 3 times. so in the csv 9 rows of data for each line

**This is done using repeats() function**

In [ ]:
def repeats():
    d1=pd.read_csv('data.csv')
    df2=pd.read_csv('dataset.csv')
    ln=len(df2['power'])
    m=ln
    print(ln)
    counter=m
    for i in range(8):
        toAdd = [df2['power'][m-1],df2['time'][m-1],df2['pressure'][m-1]]
        filename="dataset.csv"
        with open(filename, "r") as infile:
            reader = list(csv.reader(infile))
            reader.insert(counter+1, toAdd)

        with open(filename, "w", newline='') as outfile:
            writer = csv.writer(outfile)
            for line in reader:
                writer.writerow(line)
                
    for i in range(8):
        toAdd = [df2['power'][m-1],df2['time'][m-1],df2['pressure'][m-1]]
        filename="dataset-pre.csv"
        with open(filename, "r") as infile:
            reader = list(csv.reader(infile))
            reader.insert(counter+1, toAdd)

        with open(filename, "w", newline='') as outfile:
            writer = csv.writer(outfile)
            for line in reader:
                writer.writerow(line)

# (I) For the 9 readings of a single line, median is taken. it's done by take_median() function

In [ ]:
def take_median(i,idx):
    
    print(os.getcwd())
    df=pd.read_csv('dataset.csv')
    lth=len(df['ratio'])
    k=i
    median=[]

    valss=np.sort([df['ratio'][k+8],df['ratio'][k+7],df['ratio'][k+6],df['ratio'][k+5],df['ratio'][k+4],df['ratio'][k+3],
                   df['ratio'][k+2],df['ratio'][k+1],df['ratio'][k]])


    lst=[s for s in valss if str(s)!='nan']
    result=np.median(lst)
    df2=pd.read_csv('data.csv')
    df2.loc[idx,"ratio"]=result
    df2.to_csv('data.csv',index=False)
    return result